In [25]:
import wfdb
from FeatureExtraction_NB import processing,process_signal,process_qrs
import matplotlib.pyplot as plt
import pywt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [26]:
fs = 1000

patient_1 = wfdb.rdrecord('..//01.Dataset/117/s0291lre', channels=[1])
patient_2 = wfdb.rdrecord('..//01.Dataset/116/s0302lre', channels=[1])
patient_3 = wfdb.rdrecord('..//01.Dataset/105/s0303lre', channels=[1])
patient_4 = wfdb.rdrecord('..//01.Dataset/104/s0306lre', channels=[1])
patient_5 = wfdb.rdrecord('..//01.Dataset/122/s0312lre', channels=[1])
patient_6 = wfdb.rdrecord('..//01.Dataset/121/s0311lre', channels=[1])
patient_7 = wfdb.rdrecord('..//01.Dataset/150/s0287lre', channels=[1])
 

print(patient_1)


signal_1,_,_ = processing(patient_1.p_signal[: ,0])
signal_2 ,_,_= processing(patient_2.p_signal[:, 0])
signal_3 ,_,_= processing(patient_3.p_signal[:, 0])
signal_4,_,_ = processing(patient_4.p_signal[:, 0])
signal_5 ,_,_= processing(patient_5.p_signal[:, 0])
signal_6,_,_ = processing(patient_6.p_signal[:, 0])
signal_7 ,_,_= processing(patient_7.p_signal[:, 0])


print(signal_1)

time = len(signal_1)/ fs


[0.00877724 0.00944403 0.01009738 ... 0.00045572 0.00042509 0.00040158]


In [27]:
def get_Rs(signal):
    denoised_signal, y_lfiltered, window_smoothed_signal = processing(signal)
    qx, qy, sx, sy = process_signal(denoised_signal, y_lfiltered)
    qx, qy, sx, sy, Rx, Ry, qrs_on_x, qrs_on_y, qrs_off_x, qrs_off_y = process_qrs(denoised_signal, y_lfiltered, window_smoothed_signal, qx, qy, sx, sy)
    return denoised_signal,Rx

In [28]:
def non_fiducial_features_bonus(signal):
    denoised_signal,Rx = get_Rs(signal)
    nonFiducials_for_Full_signal_1 = []

    for i in range(1,len(Rx)-1):
        RR_previous = Rx[i]-Rx[i-1]
        RR_next = Rx[i+1]-Rx[i]

        nonFiducial=[]

        after_Rpeak = int (2/3*((RR_previous+RR_next)/2))

        for x in range( int(Rx[i]) , int(Rx[i])+after_Rpeak) :
            nonFiducial.append( denoised_signal[x])

        Before_Rpeak = int (1/3*((RR_previous+RR_next)/2))

        for j in range(Before_Rpeak+int(Rx[i-1]) , int(Rx[i]) ):
            nonFiducial.append( denoised_signal[j])
            
        nonFiducials_for_Full_signal_1.append(nonFiducial)
        
    list_of_non_fiducial_features_1 = []
    for i in range(len(nonFiducials_for_Full_signal_1)):
        # Define the mother wavelet
        wavelet = pywt.Wavelet('db4')

        # Define the number of levels for decomposition
        decomp_levels = 5

        # Define the list of ECG signals
        ecg_segments = np.array(nonFiducials_for_Full_signal_1[i])  # Example data

        # Decompose the signal
        decomp = pywt.wavedec(ecg_segments, wavelet, level=decomp_levels)

        CA5,CD5,CD4,CD3,CD2,CD1 =decomp 

        #only the coefficients of ECG band (1-40) use them as a feature 
        non_fiducial_feature = CA5[:40]
        non_fiducial_feature = list(non_fiducial_feature)
        if len(non_fiducial_feature) < 40:
            for i in range(40-len(non_fiducial_feature)):
                non_fiducial_feature.append(0)
        list_of_non_fiducial_features_1.append(np.array(non_fiducial_feature))
    return list_of_non_fiducial_features_1

In [29]:
non_fid1 = non_fiducial_features_bonus(signal_1)
non_fid2 = non_fiducial_features_bonus(signal_2)
non_fid3 = non_fiducial_features_bonus(signal_3)
non_fid4 = non_fiducial_features_bonus(signal_4)
non_fid5 = non_fiducial_features_bonus(signal_5)
non_fid6 = non_fiducial_features_bonus(signal_6)
non_fid7 = non_fiducial_features_bonus(signal_7)

# print(non_fid1.shape)


In [30]:
def dfs(non_fid, label):
    df = pd.DataFrame({})
    for i in range(len(non_fid)):
        df[i] = non_fid[i]
    df = df.T
    df['label'] = label
    return df

In [31]:
df1 = dfs(non_fid1, 1)
df2 = dfs(non_fid2, 2)
df3 = dfs(non_fid3, 3)
df4 = dfs(non_fid4, 4)
df5 = dfs(non_fid5, 5)
df6 = dfs(non_fid6, 6)
df7 = dfs(non_fid7, 7)


vectors = [df1,df2,df3,df4,df5,df6]

for i in range(0,6):
    #vectors[i]=vectors[i].reshape
    print(vectors[i].shape)


(127, 41)
(117, 41)
(147, 41)
(116, 41)
(125, 41)
(146, 41)


In [144]:
Data = pd.concat([df2, df3, df4,df5,df6], ignore_index=True)

In [145]:
Data

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,2.980658,3.024038,3.017857,2.908092,3.535205,1.177880,-0.485280,-0.149906,-0.188263,-0.171264,...,-0.128702,-0.043012,-0.003226,-0.067956,0.026654,0.033251,-0.005674,-0.068221,-0.040937,2
1,2.987867,3.041265,3.029060,2.916000,3.570498,1.115203,-0.377881,-0.061217,-0.084439,-0.072700,...,-0.028831,-0.041958,0.009431,0.002852,0.005286,-0.053032,-0.001953,-0.081106,0.132291,2
2,3.185943,3.232600,3.225425,3.109326,3.773703,1.279021,-0.499198,-0.136190,-0.123064,-0.166340,...,0.030073,0.124433,0.070421,0.039897,0.016395,0.020801,-0.028588,-0.034661,0.232012,2
3,2.878477,2.922891,2.916250,2.806246,3.433932,1.076079,-0.580590,-0.252698,-0.146289,-0.140102,...,-0.159162,-0.136894,-0.133894,-0.021810,0.014825,0.089966,0.029834,0.029382,0.371729,2
4,3.044320,3.088194,3.081772,2.970014,3.609101,1.211621,-0.593267,-0.202208,-0.131050,-0.051507,...,0.012589,-0.033911,-0.010958,-0.031950,0.001981,-0.026625,-0.087762,0.203998,0.340135,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,2.628148,2.660637,2.656253,2.562271,3.112962,1.030734,-0.724105,-0.333350,-0.271138,-0.197525,...,-0.052540,-0.307682,-0.542824,0.180402,2.890553,0.000000,0.000000,0.000000,0.000000,6
647,2.554109,2.590650,2.584285,2.488496,3.049859,0.930704,-0.727707,-0.313992,-0.239460,-0.178078,...,0.164970,-0.288417,-0.377252,-0.456747,1.997903,2.803819,0.000000,0.000000,0.000000,6
648,2.486747,2.518121,2.513596,2.424282,2.948648,0.964020,-0.727742,-0.334394,-0.242475,-0.148783,...,-0.090706,-0.187432,0.013737,0.562894,0.173418,-0.281696,-0.326818,-0.406287,1.888365,6
649,2.779999,2.812154,2.807831,2.715321,3.256464,1.210348,-0.689821,-0.372212,-0.289017,-0.182717,...,-0.107545,-0.178600,0.023994,0.559658,0.177610,-0.267583,-0.310984,-0.522411,1.755123,6


In [146]:
X = Data.drop('label', axis=1)
y = Data['label']

# Model

In [147]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [148]:
lr = LogisticRegression(random_state=0).fit(X, y)

# Make predictions on the test set
y_pred = lr.predict(X)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [149]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [150]:
clf = OneVsRestClassifier(SVC()).fit(X, y)

In [157]:
X_test = df1.drop('label', axis=1)
y_test = df1['label']

In [158]:
clf = LogisticRegression()
clf.fit(X, y)
y_pred = clf.predict(X_test)
probs = clf.predict_proba(X_test)
accuracy = accuracy_score(y_test,y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.0


In [159]:
print(y_pred)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [160]:
print(probs)

[[9.42173066e-02 7.42359010e-01 1.63309825e-01 6.07441612e-05
  5.31135990e-05]
 [6.87126303e-02 7.83340147e-01 1.47848467e-01 5.49628233e-05
  4.37923839e-05]
 [6.85279533e-02 8.10047919e-01 1.21261972e-01 8.73780124e-05
  7.47774371e-05]
 [1.31121518e-01 6.71527466e-01 1.97190381e-01 9.91343473e-05
  6.15006085e-05]
 [8.35619882e-02 8.05097302e-01 1.11160647e-01 1.05999031e-04
  7.40634398e-05]
 [4.70697334e-02 8.61339722e-01 9.14487883e-02 7.92511359e-05
  6.25047947e-05]
 [4.70928981e-02 8.21618423e-01 1.31173260e-01 6.94035295e-05
  4.60163165e-05]
 [9.39655865e-02 7.41165212e-01 1.64759514e-01 5.94888717e-05
  5.01985903e-05]
 [6.72365085e-02 7.72692705e-01 1.59951493e-01 6.53186808e-05
  5.39753516e-05]
 [4.78281093e-02 8.39144861e-01 1.12893365e-01 7.53544007e-05
  5.83103002e-05]
 [5.09601076e-02 8.09652396e-01 1.39254876e-01 8.77963302e-05
  4.48247061e-05]
 [6.87917914e-02 7.13141543e-01 2.17969239e-01 6.03240898e-05
  3.71020628e-05]
 [1.30488202e-01 6.36963036e-01 2.323633

In [161]:
y_pred_o = clf.predict(X_test)
y_pred_o

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [162]:
cnt = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0 , 0:0}

for i in range(len(y_pred)):
    for indx in range(0,6):
        if y_pred[i][indx] < 1.3:
            #print(y_pred[i][indx])
            if y_pred[i][indx] > 0.8:
                cnt[indx+1] += 1
        else:
            cnt[0] += 1

IndexError: invalid index to scalar variable.

In [ ]:
cnt 

{1: 0, 2: 0, 3: 0, 4: 37, 5: 0, 6: 0, 0: 83}

In [184]:
X_test = df1.drop('label', axis=1)
y_test = df1['label']

In [185]:
clf = SVC(probability=True)
clf.fit(X, y)
y_pred = clf.predict(X_test)
probs = clf.predict_proba(X_test)

In [186]:
print(y_pred)

[4 4 3 4 3 3 4 4 4 3 4 4 4 4 4 4 3 4 3 3 4 3 3 4 4 3 4 4 4 4 4 4 3 4 4 4 4
 4 4 4 4 3 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 3 4 4 4 4 4 4 3 3 3 4 3
 3 3 4 4 4 4 3 4 4 3 4 3 4 4 4 4 4 3 3 4 4 4 4 4 4 4 4 4 4 4 4 3 3 3 4 4 4
 3 3 3 4 4 3 3 4 4 3 3 4 4 4 3 4]


In [187]:
print(probs)

[[0.02978988 0.4650997  0.47242064 0.01781484 0.01487494]
 [0.0288202  0.43050097 0.50720631 0.01838828 0.01508425]
 [0.02798355 0.56725581 0.37189654 0.01859861 0.01426548]
 [0.03106762 0.51086665 0.42507109 0.01865856 0.01433608]
 [0.02761062 0.61018453 0.33115563 0.01814884 0.01290039]
 [0.02619373 0.57877314 0.36253027 0.01876935 0.01373351]
 [0.02719693 0.45633354 0.48178741 0.0195853  0.01509681]
 [0.02980787 0.46323102 0.47417534 0.01781499 0.01497077]
 [0.02884286 0.44740264 0.48965406 0.01880952 0.01529092]
 [0.02711618 0.51813285 0.42070085 0.01938544 0.01466468]
 [0.02740124 0.45988985 0.47771692 0.02034452 0.01464748]
 [0.02841006 0.35443488 0.58322005 0.01869137 0.01524364]
 [0.03305458 0.44371668 0.48579863 0.02101691 0.01641319]
 [0.02909295 0.42413085 0.5130275  0.01885451 0.01489419]
 [0.02931009 0.49102192 0.4458785  0.01930288 0.0144866 ]
 [0.03093404 0.49552855 0.43997568 0.01879947 0.01476226]
 [0.03391537 0.57500685 0.35471636 0.01987528 0.01648613]
 [0.03128081 0